# Custom plotting library fully in Python!

In [ ]:
from math import pi

import numpy as np

import branca

from ipywidgets import VBox, IntSlider

from ipycanvas import MultiCanvas, hold_canvas

In [ ]:
def init_2d_plot(x, y, color=None, scheme=branca.colormap.linear.RdBu_11, canvas=None, canvas_size=(800, 600), padding=0.1):
    if canvas is None:
        canvas = MultiCanvas(4, size=canvas_size)
    else:
        canvas.size = canvas_size

    padding_x = padding * canvas_size[0]
    padding_y = padding * canvas_size[1]

    # TODO Fix drawarea max: It should be (canvas.size - padding)
    drawarea = (drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y) = (padding_x, padding_y, canvas_size[0] - 2 * padding_x, canvas_size[1] - 2 * padding_y)

    min_x, min_y, max_x, max_y = np.min(x), np.min(y), np.max(x), np.max(y)

    dx = max_x - min_x
    dy = max_y - min_y

    # Turns a data coordinate into pixel coordinate
    scale_x = lambda x: drawarea_max_x * (x - min_x) / dx + drawarea_min_x
    scale_y = lambda y: drawarea_max_y * (1 - (y - min_y) / dy) + drawarea_min_y

    # Turns a pixel coordinate into data coordinate
    unscale_x = lambda sx: (sx - drawarea_min_x) * dx / drawarea_max_x + min_x
    unscale_y = lambda sy: (1 - ((sy - drawarea_min_y) / drawarea_max_y)) * dy + min_y

    colormap = None
    if color is not None:
        colormap = scheme.scale(np.min(color), np.max(color))        

    return canvas, drawarea, scale_x, scale_y, unscale_x, unscale_y, colormap

In [ ]:
def draw_background(canvas, drawarea, unscale_x, unscale_y):
    drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y = drawarea

    # Draw background
    canvas.fill_style = '#f7f7f7'
    canvas.global_alpha = 0.3
    canvas.fill_rect(drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y)
    canvas.global_alpha = 1

    # Draw grid and ticks
    n_lines = 10
    canvas.fill_style = 'black'
    canvas.stroke_style = '#8c8c8c'
    canvas.line_width = 1
    canvas.begin_path()

    for i in range(n_lines):
        j = i / (n_lines - 1)
        line_x = drawarea_max_x * j + drawarea_min_x
        line_y = drawarea_max_y * j + drawarea_min_y

        # Line on the y axis
        canvas.move_to(line_x, drawarea_min_y)
        canvas.line_to(line_x, drawarea_max_y + drawarea_min_y)

        # Line on the x axis
        canvas.move_to(drawarea_min_x, line_y)
        canvas.line_to(drawarea_max_x + drawarea_min_x, line_y)

        # Draw y tick
        canvas.text_align = 'right'
        canvas.text_baseline = 'middle'
        canvas.fill_text('{0:.2e}'.format(unscale_y(line_y)), drawarea_min_x * 0.95, line_y)

        # Draw x tick
        canvas.text_align = 'center'
        canvas.text_baseline = 'top'
        canvas.fill_text('{0:.2e}'.format(unscale_x(line_x)), line_x, drawarea_max_y + drawarea_min_y + drawarea_min_y * 0.05)

    canvas.stroke()
    canvas.close_path()

In [ ]:
class Scatter_plot(object):
    def __init__(self, x, y, size, color, scheme=branca.colormap.linear.RdBu_11, stroke_color='black', canvas=None):

        self.i_mark = None
        self.dragging = False
        self.x = x
        self.y = y
        self.size = size
        self.color = color
        self.stroke_color = stroke_color
        self.canvas, self.drawarea, self.scale_x, self.scale_y, self.unscale_x, self.unscale_y, self.colormap = init_2d_plot(x, y, color, scheme, canvas=canvas)
        self.show()
        
        self.canvas[3].on_mouse_down(self.mouse_down_handler)
        self.canvas[3].on_mouse_move(self.mouse_move_handler)
        self.canvas[3].on_mouse_up(self.mouse_up_handler)

    def _ipython_display_(self):
        display(self.canvas)

    def show(self, show_idx=None, skip_idx=None):
        with hold_canvas(self.canvas):
            self.canvas.clear()
            self.canvas[1].save()

            draw_background(self.canvas[0], self.drawarea, self.unscale_x, self.unscale_y)

            # Draw scatter
            self.n_marks = min(x.shape[0], y.shape[0], self.size.shape[0], self.color.shape[0])

            self.canvas[1].stroke_style = self.stroke_color
            self.canvas[2].stroke_style = self.stroke_color

            for idx in range(self.n_marks):
                if idx != skip_idx:
                    self.show_point(idx, 1)

            self.canvas[1].restore()

    def show_point(self, idx, i_canvas):
        self.canvas[i_canvas].fill_style = self.colormap(self.color[idx])

        mark_x = self.scale_x(self.x[idx])
        mark_y = self.scale_y(self.y[idx])
        mark_size = self.size[idx]

        self.canvas[i_canvas].fill_arc(mark_x, mark_y, mark_size, 0, 2 * pi)
        self.canvas[i_canvas].stroke_arc(mark_x, mark_y, mark_size, 0, 2 * pi)
                        
    def mouse_down_handler(self, pixel_x, pixel_y):
        for idx in range(self.n_marks):
            mark_x = self.x[idx]
            mark_y = self.y[idx]
            mark_size = self.size[idx]

            if (pixel_x > self.scale_x(mark_x) - mark_size and pixel_x < self.scale_x(mark_x) + mark_size and
                pixel_y > self.scale_y(mark_y) - mark_size and pixel_y < self.scale_y(mark_y) + mark_size):
                self.i_mark = idx
                self.dragging = True
                break
        
        if self.dragging:
            self.show(skip_idx=self.i_mark)
                
    def mouse_move_handler(self, pixel_x, pixel_y):
        if self.dragging:
            unscaled_x = self.unscale_x(pixel_x)
            unscaled_y = self.unscale_y(pixel_y)
            self.x[self.i_mark] = unscaled_x
            self.y[self.i_mark] = unscaled_y
            self.canvas[2].clear()
            self.show_point(self.i_mark, 2)
                    
    def mouse_up_handler(self, pixel_x, pixel_y):
        self.dragging = False

In [ ]:
def line_plot(x, y, line_color='#749cb8', line_width=2, canvas=None):
    canvas, drawarea, scale_x, scale_y, unscale_x, unscale_y, _ = init_2d_plot(x, y, canvas=canvas)

    with hold_canvas(canvas):
        canvas.clear()
        canvas[1].save()

        draw_background(canvas[0], drawarea, unscale_x, unscale_y)

        # Draw lines
        n_points = min(x.shape[0], y.shape[0])

        canvas[1].begin_path()
        canvas[1].stroke_style = line_color
        canvas[1].line_width = line_width
        canvas[1].line_join = 'bevel'
        canvas[1].line_cap = 'round'
        canvas[1].move_to(scale_x(x[0]), scale_y(y[0]))
        for idx in range(1, n_points):
            canvas[1].line_to(
                scale_x(x[idx]), scale_y(y[idx])
            )

        canvas[1].stroke()
        canvas[1].close_path()
        
        canvas[1].restore()

    return canvas

In [ ]:
def heatmap_plot(x, y, color, scheme=branca.colormap.linear.RdBu_11, canvas=None):
    canvas, drawarea, scale_x, scale_y, unscale_x, unscale_y, colormap = init_2d_plot(x, y, color, scheme, canvas=canvas)

    outof_x_bound = lambda idx: True if idx >= x.shape[0] or idx < 0 else False
    outof_y_bound = lambda idx: True if idx >= y.shape[0] or idx < 0 else False

    with hold_canvas(canvas):
        canvas.clear()
        canvas[1].save()

        draw_background(canvas[0], drawarea, unscale_x, unscale_y)

        # Draw heatmap
        n_marks = min(x.shape[0], y.shape[0])

        for x_idx in range(1, color.shape[0] - 1):
            for y_idx in range(1, color.shape[1] - 1):
                canvas[1].fill_style = colormap(color[x_idx][y_idx])

                rect_center = (scale_x(x[x_idx]), scale_y(y[y_idx]))
                neighbours_x = (scale_x(x[x_idx - 1]), scale_x(x[x_idx + 1]))
                neighbours_y = (scale_y(y[y_idx - 1]), scale_y(y[y_idx + 1]))

                rect_top_left_corner = ((neighbours_x[0] + rect_center[0]) / 2, (neighbours_y[0] + rect_center[1]) / 2)
                rect_low_right_corner = ((neighbours_x[1] + rect_center[0]) / 2, (neighbours_y[1] + rect_center[1]) / 2)

                width = rect_low_right_corner[0] - rect_top_left_corner[0]
                height = rect_low_right_corner[1] - rect_top_left_corner[1]

                canvas[1].fill_rect(
                    rect_top_left_corner[0], rect_top_left_corner[1],
                    width, height
                )

        canvas[1].restore()

    return canvas

# Scatter plot

In [ ]:
n_points = 1_000

### Scatter marks are draggable! Move the mouse while clicking on them...

In [ ]:
x = np.random.rand(n_points)
y = np.random.rand(n_points)
sizes = np.random.randint(2, 8, n_points)
colors = np.random.rand(n_points) * 10 - 2

plot = Scatter_plot(x, y, sizes, colors, branca.colormap.linear.viridis, stroke_color='white')
plot

### Because it's a Canvas, you can draw on top of it! 

In [ ]:
plot.canvas[1].stroke_style = 'red'
plot.canvas[1].line_width = 2
plot.canvas[1].stroke_rect(200, 300, 50, 100)

# Line plot

In [ ]:
x = np.linspace(0, 20, 500)
y = np.sin(x)

line_plot(x, y, line_width=3)

In [ ]:
slider = IntSlider(min=1, max=10, step=1)

x = np.linspace(-20, 20, 500)
y = np.power(x, slider.value)

power_plot = line_plot(x, y, line_color='#32a852', line_width=3)

def on_slider_change(change):
    y = np.power(x, slider.value)

    line_plot(x, y, line_color='#32a852', line_width=3, canvas=power_plot)

slider.observe(on_slider_change, 'value')

VBox((power_plot, slider))

In [ ]:
n = 1_000
x = np.linspace(0, 100, n)
y = np.cumsum(np.random.randn(n))

line_plot(x, y, line_width=3)

# Heatmap

In [ ]:
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
x_grid, y_grid = np.meshgrid(x, y)
color = np.sin(x_grid + y_grid**2) + np.cos(x_grid**2 + y_grid**2)

heatmap_plot(x, y, color, scheme=branca.colormap.linear.RdYlBu_05)